In [5]:
#Importamos libreria Pandas
import pandas as pd
import numpy as np
from osgeo import gdal
import glob
import os
from pathlib import Path

In [6]:
#definidimos la carpeta en que estan los datos
os.chdir(r"F:\UNAM\Doctorado\SIG\Clima\Estaciones_climaticas")

#creamos una lista de las estaciones que se encuentran en la carpeta
inputFiles = glob.glob("*.txt")
inputFiles

['13002.txt',
 '13004.txt',
 '13008.txt',
 '13022.txt',
 '13024.txt',
 '13027.txt',
 '13029.txt',
 '13030.txt',
 '13031.txt',
 '13033.txt',
 '13041.txt',
 '13055.txt',
 '13060.txt',
 '13061.txt',
 '13068.txt',
 '13075.txt',
 '13079.txt',
 '13082.txt',
 '13084.txt',
 '13085.txt',
 '13089.txt',
 '13091.txt',
 '13096.txt',
 '13098.txt',
 '13100.txt',
 '13111.txt',
 '13112.txt',
 '13115.txt',
 '13116.txt',
 '13117.txt',
 '13121.txt',
 '13127.txt',
 '13128.txt',
 '13129.txt',
 '13131.txt',
 '13132.txt',
 '13133.txt',
 '13138.txt',
 '13150.txt',
 '13154.txt',
 '13159.txt',
 '13160.txt',
 '13161.txt',
 '13162.txt',
 '13165.txt',
 '13167.txt',
 '13168.txt',
 '15007.txt',
 '15008.txt',
 '15013.txt',
 '15014.txt',
 '15017.txt',
 '15019.txt',
 '15020.txt',
 '15021.txt',
 '15022.txt',
 '15025.txt',
 '15028.txt',
 '15030.txt',
 '15032.txt',
 '15037.txt',
 '15039.txt',
 '15041.txt',
 '15044.txt',
 '15045.txt',
 '15047.txt',
 '15050.txt',
 '15055.txt',
 '15056.txt',
 '15057.txt',
 '15058.txt',
 '1505

## LLenado de fechas faltantes y separación de precipitación y temperatura

In [9]:
#Utilizamos un for para cambiar de estación
for i in inputFiles:
    #stem guarda el nombre sin extensión
    d=Path(i)
    f=d.stem
    
    #agregamos un print para identificar la estación con datos erroneos
    print(d)
    
    #carga la estación
    df=pd.read_csv(i,skiprows=18,sep = "\s+", skipfooter=1, engine='python')
    
    #asigna el formato fecha, se define como índice, borra columna repetida, define nombre a columnas
    df['FECHA'] = pd.to_datetime(df['FECHA'],dayfirst=True, format='%d/%m/%Y')
    df.index=df["FECHA"]
    df.drop("FECHA",axis=1, inplace=True)
    df.columns=["P(mm)","E(mm)","Tmx(°C)","Tmn(°C)"]
    
    # Define el rango de fechas, llenamos las fechas faltantes, asignamos valor Nulo, cambiamos Nulo por NaN
    idx=pd.date_range(df.index.min(),df.index.max())
    df.index = pd.DatetimeIndex(df.index)
    z=df.reindex(idx, fill_value='Nulo')
    z.index.name="Fecha"
    z=z.replace('Nulo','NaN')
    
    #z.index.strftime() sirve para definir el formato de salida de la fecha ('%d%m%Y'=01022022)
    z.index=z.index.strftime('%d/%m/%Y')
    
    #Separamos los datos en Precipitación y temperatura
    d=z["P(mm)"]
    c=z[["Tmn(°C)","Tmx(°C)"]]
    E=z['E(mm)']
    
    #Exportamos los txt separados los datos por coma
    #d.to_csv('F:/UNAM\Doctorado/SIG/Clima/Precipitacion/' +f+'P.txt', sep = ",")
    #c.to_csv('F:/UNAM\Doctorado/SIG/Clima/Temperatura/' +f+'T.txt', sep = ",")
    E.to_csv('F:/UNAM\Doctorado/SIG/Clima/Evapotranspiracion/' +f+'E.txt', sep = ",")







13002.txt
13004.txt
13008.txt
13022.txt
13024.txt
13027.txt
13029.txt
13030.txt
13031.txt
13033.txt
13041.txt
13055.txt
13060.txt
13061.txt
13068.txt
13075.txt
13079.txt
13082.txt
13084.txt
13085.txt
13089.txt
13091.txt
13096.txt
13098.txt
13100.txt
13111.txt
13112.txt
13115.txt
13116.txt
13117.txt
13121.txt
13127.txt
13128.txt
13129.txt
13131.txt
13132.txt
13133.txt
13138.txt
13150.txt
13154.txt
13159.txt
13160.txt
13161.txt
13162.txt
13165.txt
13167.txt
13168.txt
15007.txt
15008.txt
15013.txt
15014.txt
15017.txt
15019.txt
15020.txt
15021.txt
15022.txt
15025.txt
15028.txt
15030.txt
15032.txt
15037.txt
15039.txt
15041.txt
15044.txt
15045.txt
15047.txt
15050.txt
15055.txt
15056.txt
15057.txt
15058.txt
15059.txt
15061.txt
15073.txt
15074.txt
15075.txt
15077.txt
15081.txt
15083.txt
15090.txt
15093.txt
15094.txt
15095.txt
15098.txt
15099.txt
15100.txt
15101.txt
15103.txt
15106.txt
15114.txt
15115.txt
15124.txt
15125.txt
15127.txt
15129.txt
15132.txt
15135.txt
15137.txt
15145.txt
15150.txt


### Identificar errores: Fechas con valores Nulos

In [49]:
#Cargamos la estación en la cual encontramos el problema.
df=pd.read_csv(r"F:\UNAM\Doctorado\SIG\Clima\Estaciones_climaticas\9048.txt",skiprows=18,sep = "\s+", skipfooter=1, engine='python')

#establecemos el formato fecha. Agregamos coerce para que agregue nulo cuando el formato no coincida.
df['FECHA'] = pd.to_datetime(df['FECHA'],dayfirst=True, format='%d/%m/%Y', errors='coerce')

#Asignamos a idx las fechas con valor nulo
idx = df[df['FECHA'].isnull()].index

#cargamos de nuevo el txt
df=pd.read_csv(r"F:\UNAM\Doctorado\SIG\Clima\Estaciones_climaticas\29031.txt",skiprows=18,sep = "\s+", skipfooter=1, engine='python')

#identificamos el índice donde esta el error
df.iloc[idx]

#Obsevamos los datos para identificar la fecha a corregir desde el índice 
df[11674:]

,FECHA,(MM),(MM).1,(�C),(�C).1
8334,26/11/1973,Nulo,Nulo,17.5,11.2
8335,27/11/1973,Nulo,Nulo,20.5,11.6


### ValueError: cannot reindex from a duplicate axis

In [15]:
#Cargamos la estación en la cual encontramos el problema.
df=pd.read_csv(r"F:\UNAM\Doctorado\SIG\Clima\Estaciones_climaticas\9045.txt",skiprows=18,sep = "\s+", skipfooter=1, engine='python')

#Definimos formato fecha
df['FECHA'] = pd.to_datetime(df['FECHA'],dayfirst=True, format='%d/%m/%Y')

#Establecemos FECHA como índice y borramos la columna
df.index=df["FECHA"]
df.drop("FECHA",axis=1, inplace=True)

#Identifica las fechas duplicadas
df[df.index.duplicated()]



,(MM),(MM).1,(�C),(�C).1
FECHA,,,,
1989-05-01,0,Nulo,Nulo,Nulo
1992-12-01,0,Nulo,Nulo,Nulo


### Crear dataframe de rango de datos

In [3]:
#creamos dataFrame
Rango = pd.DataFrame(columns=['Estación', 'Inicial', 'Final'])

for i in inputFiles:
    #stem guarda el nombre sin extensión
    d=Path(i)
    f=d.stem
    
    #carga la estación
    df=pd.read_csv(i,skiprows=18,sep = "\s+", skipfooter=1, engine='python')
    
    #asigna el formato fecha, se define como índice, borra columna repetida, define nombre a columnas
    df['FECHA'] = pd.to_datetime(df['FECHA'],dayfirst=True, format='%d/%m/%Y')
    df.index=df["FECHA"]
    df.drop("FECHA",axis=1, inplace=True)
    
    #Llenamos 
    
    Rango = Rango.append({'Estación': f, 'Inicial':df.index.min(), 'Final':df.index.max()}, ignore_index=True)
Rango['Inicial'] = pd.to_datetime(Rango['Inicial'],dayfirst=True, format='%d/%m/%Y')
Rango['Final'] = pd.to_datetime(Rango['Final'],dayfirst=True, format='%d/%m/%Y')
    
min=Rango['Inicial'].min()
max=Rango['Final'].max()
    
    


In [4]:
Rango.to_csv('F:/UNAM\Doctorado/SIG/Clima/Precipitacion/Rango.csv', sep = ",")

# Recopilar los datos de precipitación en una tabla

In [10]:
#Creamos dataFrame con columna Fecha y la asignamos como índice
Prec = pd.DataFrame(columns=['Fecha'])
Prec.set_index('Fecha', inplace=True)
Prec

#Establecemos el rango de fecha
ix=pd.date_range(start=min, end=max, freq='D')
ix
Prec=Prec.reindex(ix)
Prec.index.name="Fecha"

#definidimos la carpeta en que estan los datos
os.chdir(r"F:\UNAM\Doctorado\SIG\Clima\Precipitacion")

#creamos una lista de las estaciones que se encuentran en la carpeta
Datos = glob.glob("*.txt")
Datos

#creamos un bucle
for i in Datos:
    d=Path(i)
    f=d.stem
    
    #print(d)
    
    #carga la estación
    df=pd.read_csv(i, engine='python')
    df['Fecha'] = pd.to_datetime(df['Fecha'],dayfirst=True, format='%d/%m/%Y')
    df.index=df["Fecha"]
    df.drop("Fecha",axis=1, inplace=True)
    
    Prec=pd.merge(Prec, df,how='left', left_index=True, right_index=True)
    Prec.rename({'P(mm)': f}, axis=1, inplace=True) 
    
#Prec.to_csv('F:/UNAM\Doctorado/SIG/Clima/Precipitacion/Total.csv', sep = ",")




    
    





In [19]:
Prec_Y=Prec.resample(rule='Y').sum()
Prec_sum=Prec_Y.mean()
Prec_sum.to_csv('F:/UNAM\Doctorado/SIG/Clima/Precipitacion/sum.csv', sep = ",")

In [70]:
Prec.shape

(52025, 187)

In [72]:
Prec.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52025 entries, 1877-03-06 to 2019-08-13
Freq: D
Columns: 187 entries, 13002P to 9071P
dtypes: float64(187)
memory usage: 76.6 MB


In [73]:
Prec.describe()

,13002P,13004P,13008P,13022P,13024P,13027P,13029P,13030P,13031P,13033P,...,9022P,9029P,9032P,9036P,9041P,9045P,9048P,9051P,9068P,9071P
count,10068.000000,2179.000000,16495.000000,26689.000000,12786.000000,15824.000000,17564.000000,15293.000000,14584.000000,16716.000000,...,18466.000000,23160.000000,22110.000000,16904.000000,16597.000000,22792.000000,51476.000000,16469.000000,12027.000000,10891.000000
mean,1.823679,3.978247,1.491064,0.308876,1.641850,1.768213,1.959890,1.987844,1.660155,1.661013,...,3.602410,1.731934,2.027802,1.971575,2.689281,1.717467,2.009968,1.551309,1.175505,1.988697
std,5.343156,9.515685,4.834248,2.268262,4.841227,5.411553,5.820881,5.764822,5.166467,5.175233,...,8.026609,5.090666,5.404588,5.565276,6.932433,4.981375,5.535295,4.653498,4.094088,5.316575
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.100000,0.500000,0.000000,0.000000,0.500000,0.400000,0.300000,0.400000,0.000000,0.000000,...,3.000000,0.500000,0.800000,0.500000,1.000000,0.500000,0.800000,0.000000,0.000000,1.000000
max,84.500000,81.500000,89.900000,105.800000,104.000000,106.200000,140.000000,100.500000,100.000000,111.000000,...,115.000000,84.500000,93.000000,95.000000,95.000000,72.000000,80.600000,66.000000,117.000000,91.200000


In [8]:
#Datos por días
count=Prec.count(axis=1)
count.to_csv("Datos_dia.csv", sep=',')

#Datos por estación
count_d=Prec.count(axis=0)
count_d.to_csv("Datos_Estacion.csv", sep=',')


Fecha
1877-03-06    1
1877-03-07    1
1877-03-08    1
1877-03-09    1
1877-03-10    1
             ..
2019-08-09    5
2019-08-10    5
2019-08-11    5
2019-08-12    5
2019-08-13    0
Freq: D, Length: 52025, dtype: int64